# Project 3, BART Line Routing Information

# BART Map

![Bart Map](bart_map.png)

In [1]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

import pandas as pd

import neo4j

In [2]:
## function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

In [5]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

# 3.1 Drop the stations table if it exists


In [6]:
connection.rollback()

query = """

drop table if exists stations;

"""

cursor.execute(query)

connection.commit()


# 3.2 Drop the lines table if it exists

In [7]:
connection.rollback()

query = """

drop table if exists lines;

"""

cursor.execute(query)

connection.commit()


# 3.3 Drop the travel times table if it exists

In [14]:
connection.rollback()

query = """

drop table if exists travel_times;

"""

cursor.execute(query)

connection.commit()


# 3.4 Drop the degree_centrality table if it exists

In [9]:
connection.rollback()

query = """

drop table if exists degree_centrality;

"""

cursor.execute(query)

connection.commit()

# 3.5 Create the stations table

In [10]:
connection.rollback()

query = """

create table stations (
  station varchar(32),
  latitude numeric(9,6),
  longitude numeric(9,6),
  transfer_time numeric(3),
  primary key (station)
);

"""

cursor.execute(query)

connection.commit()

# 3.6 Create the lines table

In [12]:
connection.rollback()

query = """

create table lines (
  line varchar(6),
  sequence numeric(2),
  station varchar(32),
  primary key (line, sequence)
);

"""

cursor.execute(query)

connection.commit()

# 3.7 Create the travel times table

In [15]:
connection.rollback()

query = """

create table travel_times (
  station_1 varchar(32),
  station_2 varchar(32),
  travel_time numeric(3),
  primary key (station_1, station_2)
);

"""

cursor.execute(query)

connection.commit()

# 3.8 Create the degree_centrality table

In [17]:
connection.rollback()

query = """

create table degree_centrality (
  unnamed varchar(32),
  zip varchar(32),
  n_lat numeric(9,6),
  n_lon numeric(9,6),
  customer_id varchar(32),
  first_name varchar(32),
  last_name varchar(32),
  street varchar(32),
  louvein_community varchar(32),
  city varchar(32),
  central_node numeric(1),
  primary key (unnamed)
);

"""

cursor.execute(query)

connection.commit()

DuplicateTable: relation "degree_centrality" already exists


# 3.9 Display the file stations.csv 

In [18]:
my_read_csv_file("stations.csv", limit=10)

['station', 'latitude', 'longitude', 'transfer_time']
['12th Street', '37.803608', '-122.272006', '282']
['16th Street Mission', '37.764847', '-122.420042', '287']
['19th Street', '37.807869', '-122.26898', '67']
['24th Street Mission', '37.752', '-122.4187', '277']
['Antioch', '37.996281', '-121.783404', '0']
['Ashby', '37.853068', '-122.269957', '299']
['Balboa Park', '37.721667', '-122.4475', '48']
['Bay Fair', '37.697', '-122.1265', '63']
['Berryessa', '37.368361', '-121.874655', '288']

Printed  10 lines of  51 total lines.


# 3.10 Display the file lines.csv 

In [19]:
my_read_csv_file("lines.csv", limit=10)

['line', 'sequence', 'station']
['blue', '1', 'Dublin']
['blue', '2', 'West Dublin']
['blue', '3', 'Castro Valley']
['blue', '4', 'Bay Fair']
['blue', '5', 'San Leandro']
['blue', '6', 'Coliseum']
['blue', '7', 'Fruitvale']
['blue', '8', 'Lake Merritt']
['blue', '9', 'West Oakland']

Printed  10 lines of  115 total lines.


# 3.11 Display the file travel_times.csv 

In [20]:
my_read_csv_file("travel_times.csv", limit=10)

['station_1', 'station_2', 'travel_time']
['12th Street', '19th Street', '120']
['12th Street', 'Lake Merritt', '180']
['12th Street', 'West Oakland', '300']
['16th Street Mission', '24th Street Mission', '120']
['16th Street Mission', 'Civic Center', '180']
['19th Street', 'MacArthur', '180']
['24th Street Mission', 'Glen Park', '180']
['Antioch', 'Pittsburg Center', '420']
['Ashby', 'Downtown Berkeley', '180']

Printed  10 lines of  52 total lines.


# 3.12 Display the file degree_centrality.csv 

In [21]:
my_read_csv_file("degree_centrality.csv", limit=15)

['unnamed', 'zip', 'n_lat', 'n_lon', 'customer_id', 'first_name', 'last_name', 'street', 'louvein_community', 'city', 'central_node']
['4', '94005', '37.68234820825750', '-122.40670651156800', '7158', 'Malissa', 'Latek', '8346_Fieldstone_Park', '6860', 'Brisbane', '0']
['9', '94005', '37.68321598513340', '-122.40961149963500', '7189', 'Kaja', 'Killiner', '2122_Eagle_Crest_Plaza', '6860', 'Brisbane', '0']
['10', '94005', '37.682674402903400', '-122.41440936175300', '7188', 'Trever', 'Wais', '13540_Manitowish_Lane', '6860', 'Brisbane', '0']
['12', '94005', '37.68336480264470', '-122.40701915414600', '7186', 'Lucius', 'Gawne', '6586_Dorton_Alley', '6860', 'Brisbane', '0']
['17', '94005', '37.68602971854240', '-122.41062157360500', '7181', 'Fraser', 'Vasser', '6313_Warbler_Circle', '6860', 'Brisbane', '0']
['18', '94005', '37.68720275088050', '-122.4099590326040', '7180', 'Candie', 'Dayne', '807_Roth_Road', '6860', 'Brisbane', '0']
['25', '94005', '37.682011692210800', '-122.40387815241800

# 3.13 Load stations data into database table

In [22]:
connection.rollback()

query = """

copy stations
from '/user/projects/project-3-jrodriguez-jr/exercise/stations.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

# 3.14 Load lines data into database table

In [23]:
connection.rollback()

query = """

copy lines
from '/user/projects/project-3-jrodriguez-jr/exercise/lines.csv' delimiter ',' NULL '' csv header;


"""

cursor.execute(query)

connection.commit()

# 3.15 Load travel_times data into database table

In [24]:
connection.rollback()

query = """

copy travel_times
from '/user/projects/project-3-jrodriguez-jr/exercise/travel_times.csv' delimiter ',' NULL '' csv header;


"""

cursor.execute(query)

connection.commit()

# 3.16 Load degree_centrality data into database table

In [25]:
#project 3
connection.rollback()

query = """

copy degree_centrality
from '/user/projects/project-3-jrodriguez-jr/exercise/degree_centrality.csv' delimiter ',' NULL '' csv header;


"""

cursor.execute(query)

connection.commit()

# 3.17 Verify the stations loaded correctly

In [26]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from stations
order by station

"""

df_stations = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df_stations

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0
5,Ashby,37.853068,-122.269957,299
6,Balboa Park,37.721667,-122.447500,48
7,Bay Fair,37.697000,-122.126500,63
8,Berryessa,37.368361,-121.874655,288
9,Castro Valley,37.690748,-122.075679,0


# 3.18 Verify the lines loaded correctly

In [32]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from lines
order by line, sequence

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,sequence,station
0,blue,1,Dublin
1,blue,2,West Dublin
2,blue,3,Castro Valley
3,blue,4,Bay Fair
4,blue,5,San Leandro
...,...,...,...
109,yellow,23,Daly City
110,yellow,24,Colma
111,yellow,25,South San Francisco
112,yellow,26,San Bruno


# 3.19 Verify the travel times loaded correctly

In [33]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from travel_times
order by station_1, station_2

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station_1,station_2,travel_time
0,12th Street,19th Street,120
1,12th Street,Lake Merritt,180
2,12th Street,West Oakland,300
3,16th Street Mission,24th Street Mission,120
4,16th Street Mission,Civic Center,180
5,19th Street,MacArthur,180
6,24th Street Mission,Glen Park,180
7,Antioch,Pittsburg Center,420
8,Ashby,Downtown Berkeley,180
9,Ashby,MacArthur,240


# 3.20 Verify the degree_centrality table loaded correctly

In [34]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from degree_centrality
order by unnamed

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,unnamed,zip,n_lat,n_lon,customer_id,first_name,last_name,street,louvein_community,city,central_node
0,10,94005,37.682674,-122.414409,7188,Trever,Wais,13540_Manitowish_Lane,6860,Brisbane,0
1,12,94005,37.683365,-122.407019,7186,Lucius,Gawne,6586_Dorton_Alley,6860,Brisbane,0
2,17,94005,37.686030,-122.410622,7181,Fraser,Vasser,6313_Warbler_Circle,6860,Brisbane,0
3,18,94005,37.687203,-122.409959,7180,Candie,Dayne,807_Roth_Road,6860,Brisbane,0
4,25,94005,37.682012,-122.403878,7173,Joshuah,MacFadzan,135_Di_Loreto_Park,6860,Brisbane,0
5,26,94005,37.682657,-122.411316,7172,Francois,Farish,1_Texas_Trail,6860,Brisbane,1
6,27,94005,37.681886,-122.416236,7171,Byrann,Feechum,5_Tony_Plaza,6860,Brisbane,0
7,30,94005,37.682597,-122.398976,7168,Ashly,Lindup,47_Fieldstone_Parkway,6860,Brisbane,0
8,32,94005,37.679975,-122.412504,7166,Riva,Schoolcroft,64581_Atwood_Junction,6860,Brisbane,0
9,34,94005,37.682768,-122.412029,7164,Mellie,Berriball,85524_Carey_Lane,6860,Brisbane,0


# 3.21 Find the nearest BART Station

In [62]:
from math import radians, sin, cos, acos

# louvein community: 6860, (4, 37.682348, -122.406707, Malissa Latek, 8346_Fieldstone_Park)
df_temp = pd.DataFrame(columns=['station','distance'])
z = len(df_stations.index)
i = 0
x = 0
y = 0

while i <= z-1:
    mlat = 37.682348
    mlon = -122.4067
    plat = df_stations.iloc[x,1]
    plon = df_stations.iloc[y,2]
    l = 371.01 * acos(sin(mlat)*sin(plat) + cos(mlat)*cos(plat)*cos(mlon - plon))
    df_temp = df_temp.append({'station':df_stations.iloc[x,0],'distance':l}, ignore_index = True)
    i+=1
    x+=1
    y+=1

louvein_station = df_stations.iloc[df_temp['distance'].idxmin(),0]
print('The nearest BART Station from the Louvein Community is', louvein_station)

The nearest BART Station from the Louvein Community is South San Francisco


# 3.1.22 Query the lines served by the Initial BART Station: Milpitas

In [38]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select station, line
from lines
where station = 'Milpitas'
order by station, line

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,Milpitas,green
1,Milpitas,orange


# 3.23 Query the list of all possible line transfers at the Initial BART Station and the transfer times

In [39]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
where a.station = 'Milpitas'
order by 1, 2, 3

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,Milpitas,green,orange,292
1,Milpitas,orange,green,292


# 3.24 Query the list of all segments between the Initial BART Station and its adjoining stations

In [40]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
where (a.station = 'Milpitas' or b.station = 'Milpitas')
order by line, from_station, to_station

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,from_station,to_station,travel_time
0,green,Berryessa,Milpitas,300
1,green,Milpitas,Warm Springs,540
2,orange,Milpitas,Berryessa,300
3,orange,Warm Springs,Milpitas,540


# 3.25 Create a graph database in Neo4j for the BART system

In [41]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [42]:
session = driver.session(database="neo4j")

In [43]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [44]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [45]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [46]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [47]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [48]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [49]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

# 3.26 Wipe out the Neo4j database

In [50]:
my_neo4j_wipe_out_database()

# 3.27 Verify the number of nodes and relationships

In [51]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


# 3.28 Query the list of stations and create the departure and arrival nodes in the graph

In [52]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)

# 3.29 Verify the number of nodes and relationships

In [53]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


# 3.30 Query the list of stations and the lines they serve, create line nodes, and create relationships between the line nodes and the departure and arrival nodes with weight 0

In [54]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)

# 3.31 Verify the number of nodes and relationships

In [55]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


# 3.32 Query the list of all possible line transfers and the transfer times, create a relationship for each transfer with the transfer time as the weight

In [56]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)

# 3.33 Verify the number of nodes and relationships

In [57]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


# 3.34 Query the list of all segments between each station and its adjoining stations, create a relationship for each segment both ways

In [58]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)

# 3.35 Verify the number of nodes and relationships

In [59]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


# 3.36 Verify the shortest path between Initial BART Station and Louvein Community

In [60]:
print('The nearest BART Station from the Louvein Community is', louvein_station)

The nearest BART Station from the Louvein Community is South San Francisco


In [61]:
my_neo4j_shortest_path('depart Milpitas', 'arrive South San Francisco')


--------------------------------
   Total Cost:  5328
   Minutes:  88.8
--------------------------------
depart Milpitas, 0, 0
green Milpitas, 0, 0
green Warm Springs, 540, 540
green Fremont, 360, 900
green Union City, 300, 1200
green South Hayward, 300, 1500
green Hayward, 240, 1740
green Bay Fair, 240, 1980
green San Leandro, 240, 2220
green Coliseum, 240, 2460
green Fruitvale, 240, 2700
green Lake Merritt, 300, 3000
green West Oakland, 360, 3360
green Embarcadero, 420, 3780
green Montgomery Street, 60, 3840
green Powell Street, 120, 3960
green Civic Center, 60, 4020
green 16th Street Mission, 180, 4200
green 24th Street Mission, 120, 4320
green Glen Park, 180, 4500
green Balboa Park, 120, 4620
red Balboa Park, 48, 4668
red Daly City, 240, 4908
red Colma, 240, 5148
red South San Francisco, 180, 5328
arrive South San Francisco, 0, 5328
